In [2]:
!pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import re

print("Всі бібліотеки працюють!")

KeyboardInterrupt: 

In [ ]:
#1
import urllib.request
from datetime import datetime

regions = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька", 
    5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська", 
    9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська", 
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівненська", 
    17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська", 
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська", 
    25: "Республіка Крим"
}

for region_id, region_name in regions.items():
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean"
    
    filename = f"VHI_{region_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    response = urllib.request.urlopen(url)
    with open(filename, 'wb') as file:
        file.write(response.read())

    print(f"Завантажено: {filename}")

In [ ]:
#2
filename = "VHI_Черкаська_20250410_151141.csv"
df = pd.read_csv(filename, index_col=False, header=1)

print(df.head())

In [ ]:
# 3, Переконуємося, що назви колонок коректні
df.columns = df.columns.str.replace(r'<.*?>', '', regex=True).str.strip()

if "VHI" in df.columns:
    print("Колонка VHI є!")
    df_cleaned = df[df["VHI"] != -1].copy()  # .copy() уникає Warning

    print(df[df["year"] == 1982][["year", "VHI"]].head(10))
    df_cleaned.loc[:, "region"] = "Черкаська"

    print(df_cleaned.head())
else:
    print("Помилка: Колонка VHI відсутня!")


In [ ]:
# 4, Вибираємо рік для аналізу
year_to_analyze = 1982
df_year = df_cleaned[df_cleaned["year"] == year_to_analyze]

print(df_year.head())
if df_year.empty:
    print("Дані за цей рік відсутні!")
else:
    mean_vhi = df_year["VHI"].mean()
    min_vhi = df_year["VHI"].min()
    max_vhi = df_year["VHI"].max()

    min_vhi_week = df_year[df_year["VHI"] == min_vhi]["week"].values[0]
    max_vhi_week = df_year[df_year["VHI"] == max_vhi]["week"].values[0]

    print(f"Аналіз року {year_to_analyze}")
    print(f"Середнє VHI: {mean_vhi:.2f}")
    print(f"Мінімальне VHI: {min_vhi:.2f} (Тиждень {min_vhi_week})")
    print(f"Максимальне VHI: {max_vhi:.2f} (Тиждень {max_vhi_week})")


In [ ]:
#5
import seaborn as sns
import matplotlib.pyplot as plt

year_to_analyze = 1982
df_year = df_cleaned[df_cleaned["year"] == year_to_analyze]

if df_year.empty:
    print(f"Дані за {year_to_analyze} рік відсутні!")
else:
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df_year, x="week", y="VHI", marker="o", linewidth=2.5, color="blue")

    plt.title(f"Динаміка VHI за {year_to_analyze} рік", fontsize=14)
    plt.xlabel("Тиждень", fontsize=12)
    plt.ylabel("VHI", fontsize=12)
    plt.xticks(range(1, 53, 2))  # Робимо підписи тижнів кожні 2 тижні
    plt.grid(True, linestyle="--", alpha=0.6)
    
    plt.show()

In [ ]:
# 6, Параметри вибірки
start_year = 1982
end_year = 2024
selected_regions = ["Київська", "Львівська", "Одеська"]

csv_files = [f for f in os.listdir() if f.endswith('.csv') and f.startswith("VHI_")]

dfs = []
for file in csv_files:
    try:
        df = pd.read_csv(file, index_col=False, header=1)
        if df.empty or df.shape[1] < 5:
            print(f"Пропущено файл {file}")
            continue
        region_match = re.search(r'VHI_(.*?)_', file)
        region_name = region_match.group(1) if region_match else "Невідомо"
        df["region"] = region_name
        dfs.append(df)
    except Exception as e:
        print(f"Помилка при зчитуванні файлу {file}: {e}")

df_cleaned = pd.concat(dfs, ignore_index=True)
df_cleaned = df_cleaned.drop_duplicates()

df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.str.contains('^Unnamed')]
df_cleaned = df_cleaned.dropna(how='all')

df_cleaned.columns = df_cleaned.columns.str.replace(r'<.*?>', '', regex=True).str.strip()
df_cleaned["year"] = df_cleaned["year"].astype(str).str.extract(r'(\d{4})')
df_cleaned["year"] = pd.to_numeric(df_cleaned["year"], errors="coerce")
df_cleaned = df_cleaned.dropna(subset=["year"])
df_cleaned["year"] = df_cleaned["year"].astype(int)
df_cleaned["region"] = df_cleaned["region"].astype(str).str.replace(r'<.*?>', '', regex=True).str.strip()

df_selected = df_cleaned[
    (df_cleaned["year"] >= start_year) & 
    (df_cleaned["year"] <= end_year) & 
    (df_cleaned["region"].isin(selected_regions))
]

print(f"\nВибірка VHI за роки {start_year}-{end_year} для областей: {', '.join(selected_regions)}")
print(df_selected.head(10))

df_selected.to_csv("VHI_selected_years_regions.csv", index=False, encoding="utf-8-sig")
print("\nДані збережено у файл VHI_selected_years_regions.csv")

In [ ]:
# 7, Параметри аналізу
drought_threshold = 15  # Межа для посухи (VHI ≤ 15)
affected_threshold = 5  # Мінімальна кількість областей (20% від 25)

df_cleaned["region"] = df_cleaned["region"].astype(str).str.strip()
drought_data = df_cleaned[df_cleaned["VHI"] <= 15]

drought_summary = drought_data.groupby("year")["region"].nunique().reset_index()
drought_summary.columns = ["year", "affected_regions"]

print(drought_summary)

print("Роки з ≥5 областями з посухою:")
print(drought_summary[drought_summary["affected_regions"] >= 5])
severe_droughts = drought_summary[drought_summary["affected_regions"] >= affected_threshold]


print("Роки з екстремальними посухами (VHI ≤ 15, постраждало ≥5 областей):")
print(severe_droughts)

for year in severe_droughts["year"]:
    regions_affected = drought_data[drought_data["year"] == year]["region"].unique()
    print(f"{year}: {len(regions_affected)} областей під посухою - {', '.join(regions_affected)}")

severe_droughts.to_csv("VHI_severe_droughts.csv", index=False, encoding="utf-8-sig")
print("Дані збережено у VHI_severe_droughts.csv")
